### Imports + Support functions

In [ ]:
import random
import math
import tqdm
import numpy as np
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.datasets import make_moons
import matplotlib.pyplot as plt

In [ ]:
def aslist(value):
    # Converts iterables to lists and single values to a single-element lists
    try:
        return list(value)
    except:
        return (value,)

def gradient_descent(model, X, y, lr=1e-6, steps=250):
    losses = []
    progress = tqdm.trange(steps)
    for i in progress:
        loss, g = model.nll_and_grad(X, y)
        if isinstance(model.weights, np.ndarray):
            model.weights = model.weights - lr * g
        else:
            model.weights = [wi - lr * gi for (wi, gi) in zip(aslist(model.weights), aslist(g))]
        accuracy = model.accuracy(X, y)

        losses.append(loss)
        progress.set_description('Loss %.2f, accuracy: %.2f' % (loss, accuracy))
        
    return losses

def plot_boundary(model, X, y):
    xrange = (-X[:, 0].min() + X[:, 0].max()) / 10
    yrange = (-X[:, y].min() + X[:, y].max()) / 10
    feature_1, feature_2 = np.meshgrid(
        np.linspace(X[:, 0].min() - xrange, X[:, 0].max() + xrange),
        np.linspace(X[:, 1].min() - yrange, X[:, 1].max() + yrange)
    )
    grid = np.vstack([feature_1.ravel(), feature_2.ravel()]).T
    y_pred = np.reshape(model.predict(grid), feature_1.shape)
    display = DecisionBoundaryDisplay(
        xx0=feature_1, xx1=feature_2, response=y_pred
    )
    display.plot()
    display.ax_.scatter(
        X[:, 0], X[:, 1], c=y.flatten(), edgecolor="black"
    )
    plt.show()

### Reverse-mode automatic differentiation

In [ ]:
class AutogradValue:
    '''
    Base class for automatic differentiation operations. Represents variable delcaration.
    Subclasses will overwrite func and grads to define new operations.

    Properties:
        parents (list): A list of the inputs to the operation, may be AutogradValue or float
        args    (list): A list of raw values of each input (as floats)
        grad    (float): The derivative of the final loss with respect to this value (dL/da)
        value   (float): The value of the result of this operation
    '''

    def __init__(self, *args):
        self.parents = list(args)
        self.args = [arg.value if isinstance(arg, AutogradValue) else arg for arg in self.parents]
        self.grad = 0.
        self.value = self.forward_pass()

    def func(self, input):
        '''
        Compute the value of the operation given the inputs.
        For declaring a variable, this is just the identity function (return the input).

        Args:
            input (float): The input to the operation
        Returns:
            value (float): The result of the operation
        '''
        return input

    def grads(self, *args):
        '''
        Compute the derivative of the operation with respect to each input.
        In the base case the derivative of the identity function is just 1. (da/da = 1).

        Args:
            input (float): The input to the operation
        Returns:
            grads (tuple): The derivative of the operation with respect to each input
                            Here there is only a single input, so we return a length-1 tuple.
        '''
        return (1,)
    
    def forward_pass(self):
        # Calls func to compute the value of this operation 
        return self.func(*self.args)
    
    def __repr__(self):
        # Python magic function for string representation.
        return str(self.value)


In [ ]:
class _add(AutogradValue):
    # Addition operator (a + b)
    def func(self, a, b):
        return a + b
    
    def grads(self, a, b):
        return 1., 1.
    
class _neg(AutogradValue):
    # Negation operator (-a)
    def func(self, a):
        return -a
    
    def grads(self, a):
        return (-1.,)

class _sub(AutogradValue):
    # Subtraction operator (a - b)
    def func(self, a, b):
        return a - b
    
    def grads(self, a, b):
        return 1., -1.

class _mul(AutogradValue):
    # Multiplication operator (a * b)
    def func(self, a, b):
        return a * b
    
    def grads(self, a, b):
        return (b, a)

class _div(AutogradValue):
    # Division operator (a / b)
    def func(self, a, b):
        return a / b
    
    def grads(self, a, b):
        return (1. / b, -a / b**2.)
    
class _exp(AutogradValue):
    # Exponent operator (e^a, or exp(a))
    def func(self, a):
        return np.exp(a)
    
    def grads(self, a):
        return (np.exp(a),)
    
class _log(AutogradValue):
    # (Natural) log operator (log(a))
    def func(self, a):
        return np.log(a)
    
    def grads(self, a):
        return (1. / a, )

In [ ]:
def exp(a):
    return _exp(a) if isinstance(a, AutogradValue) else math.exp(a)

def log(a):
    return _log(a) if isinstance(a, AutogradValue) else math.log(a)


AutogradValue.exp = lambda a: _exp(a)
AutogradValue.log = lambda a: _log(a)
AutogradValue.__add__ = lambda a, b: _add(a, b)
AutogradValue.__radd__ = lambda a, b: _add(b, a)
AutogradValue.__sub__ = lambda a, b: _sub(a, b)
AutogradValue.__rsub__ = lambda a, b: _sub(b, a)
AutogradValue.__neg__ = lambda a: _neg(a)
AutogradValue.__mul__ = lambda a, b: _mul(a, b)
AutogradValue.__rmul__ = lambda a, b: _mul(b, a)
AutogradValue.__truediv__ = lambda a, b: _div(a, b)
AutogradValue.__rtruediv__ = lambda a, b: _div(b, a)

In [ ]:
def graph_print(a):
    if isinstance(a, AutogradValue): 
        print(a.value)
        [graph_print(p) for p in a.parents]
    else:
        print(a)

a = AutogradValue(5.)
b = AutogradValue(2.)
c = log((a + 5) * b)
graph_print(c)

In [ ]:
# Backward Pass function
def backward_pass(self):

    grads = self.grads(*self.args)
    
    for p in self.parents:
        if isinstance(p, AutogradValue):
            p.grad += self.grad * grads[self.parents.index(p)]

AutogradValue.backward_pass = backward_pass

def backward(self):
    # We call backward on the loss, so dL/dL = 1
    self.grad = 1.

    visited = [self]
    queue = [self]

    while queue:
        node = queue.pop()

        for p in node.parents:
            if isinstance(p, AutogradValue):
                try:
                    visited.remove(p)
                except ValueError:
                    pass
                visited.append(p)
                queue.append(p)

    for node in visited:
        node.backward_pass()

AutogradValue.backward = backward

In [ ]:
# Test backward
a = AutogradValue(5)
b = AutogradValue(2)

L = -log(5 *b + a)
L.backward()
print(a.grad, b.grad) # -0.06666666666666667 -0.3333333333333333

a = np.array([AutogradValue(5), AutogradValue(2)])
L = np.dot(a, a)
L.backward()
print(a[0].grad, a[1].grad) # 10.0 4.0

### Implementing a Neural Network

In [ ]:
def wrap_array(a):
    '''
    Wraps the elements of an array with AutogradValue

    Args:
        a (array of float): The array to wrap
    Returns:
        g (array of AutogradValue): An array g, such that g[i,j] = AutogradValue(a[i,j])
    '''
    g = []
    for i in a:
        g += [[AutogradValue(j) for j in i]]

    return np.array(g)

def unwrap_gradient(a):
    '''
    Unwraps the gradient of an array with AutogradValues

    Args:
        a (array of AutogradValue): The array to unwrap
    Returns:
        g (array of float): An array g, such that g[i,j] = a[i,j].grad
    '''
    g = []
    for i in a:
        g += [[j.grad for j in i]]

    return np.array(g)

In [ ]:
def sigmoid(x):
    # Computes the sigmoid function
    return 1. / (1. + np.exp(-x))

class LogisticRegression:
    def __init__(self, dims):
        '''
        Args:
            dims (int): d, the dimension of each input
        '''
        self.weights = np.zeros((dims + 1, 1))

    def prediction_function(self, X, w):
        '''
        Get the result of our base function for prediction (i.e. x^t w)

        Args:
            X (array): An N x d matrix of observations.
            w (array): A (d+1) x 1 vector of weights.
        Returns:
            pred (array): A length N vector of f(X).
        '''
        X = np.pad(X, ((0,0), (0,1)), constant_values=1.)
        return np.dot(X, w)

    def predict(self, X):
        '''
        Predict labels given a set of inputs.

        Args:
            X (array): An N x d matrix of observations.
        Returns:
            pred (array): An N x 1 column vector of predictions in {0, 1}
        '''
        return (self.prediction_function(X, self.weights) > 0)
    
    def predict_probability(self, X):
        '''
        Predict the probability of each class given a set of inputs

        Args:
            X (array): An N x d matrix of observations.
        Returns:
            probs (array): An N x 1 column vector of predicted class probabilities
        '''
        return sigmoid(self.prediction_function(X, self.weights))

    def accuracy(self, X, y):
        '''
        Compute the accuracy of the model's predictions on a dataset

        Args:
            X (array): An N x d matrix of observations.
            y (array): A length N vector of labels.
        Returns:
            acc (float): The accuracy of the classifier
        '''
        y = y.reshape((-1, 1))
        return (self.predict(X) == y).mean()

    def nll(self, X, y, w=None):
        '''
        Compute the negative log-likelihood loss.

        Args:
            X (array): An N x d matrix of observations.
            y (array): A length N vector of labels.
            w (array, optional): A (d+1) x 1 matrix of weights.
        Returns:
            nll (float): The NLL loss
        '''
        if w is None:
            w = self.weights

        y = y.reshape((-1, 1))
        xw = self.prediction_function(X, w)
        py = sigmoid((2 * y - 1) * xw)
        return -(np.log(py)).sum()
    
    def nll_gradient(self, X, y):
        '''
        Compute the gradient of the negative log-likelihood loss.

        Args:  
            X (array): An N x d matrix of observations.
            y (array): A length N vector of labels.
        Returns:
            grad (array): A length (d + 1) vector with the gradient
        '''
        y = y.reshape((-1, 1))
        xw = self.prediction_function(X, self.weights)
        py = sigmoid((2 * y - 1) * xw)
        grad = ((1 - py) * (2 * y - 1)).reshape((-1, 1)) * np.pad(X, [(0,0), (0,1)], constant_values=1.)
        return -np.sum(grad, axis=0)
    
    def nll_and_grad_no_autodiff(self, X, y):
        # Compute nll_and_grad without automatic diferentiation
        return self.nll(X, y), self.nll_gradient(X, y)

    def nll_and_grad(self, X, y):
        W = wrap_array(self.weights)
        L = LogisticRegression.nll(self, X, y, W)
        L.backward()

        grad = unwrap_gradient(W)

        return L.value, grad  
    

In [ ]:
class NeuralNetwork(LogisticRegression):
    def __init__(self, dims, hidden_sizes=[]):
        ## YOUR CODE HERE
        self.weights = [np.random.normal(scale=1., size=(hidden_sizes[0], dims))]

        for i in range(0, len(hidden_sizes) - 1):
            shape = (hidden_sizes[i], hidden_sizes[i + 1])
            self.weights += [np.random.normal(scale=1., size=shape)]

        self.weights += [(np.random.normal(scale=1., size=(hidden_sizes[-1], 1)))]

In [ ]:
def prediction_function(self, X, w):
    '''
    Get the result of our base function for prediction (i.e. x^t w)

    Args:
        X (array): An N x d matrix of observations.
        w (list of arrays): A list of weight matrices
    Returns:
        pred (array): An N x 1 matrix of f(X).
    '''
    ## YOUR CODE HERE
    neurons = sigmoid(np.dot(X, np.array(w[0]).T))
    
    for i in range(1, len(w) - 1):
        neurons = sigmoid(np.dot(neurons, np.array(w[i]).T))

    pred = np.dot(neurons, w[-1])
    return pred.reshape(-1, 1)

NeuralNetwork.prediction_function = prediction_function

def nll_and_grad(self, X, y):
    '''
    Get the negative log-likelihood loss and its gradient

    Args:
        X (array): An N x d matrix of observations.
        y (array): A length N vector of labels
    Returns:
        nll (float): The negative log-likelihood
        grads (list of arrays): A list of the gradient of the nll with respect
                                to each value in self.weights.
    '''
    ## YOUR CODE HERE
    grad = []
    W = [wrap_array(w) for w in self.weights]
    
    L = NeuralNetwork.nll(self, X, y, W)
    L.backward()

    grad = [unwrap_gradient(w) for w in W]

    return L.value, grad  

NeuralNetwork.nll_and_grad = nll_and_grad

In [ ]:
# Train neural network!
X, y = make_moons(100, noise=0.1)
model = NeuralNetwork(2, [5, 5])
gradient_descent(model, X, y, lr=3e-2, steps=250)

print('Model accuracy: %.3f' % model.accuracy(X, y))
plot_boundary(model, X, y) 

### Forward-mode automatic differentiation

In [ ]:
class ForwardValue(AutogradValue):
    def __init__(self, *args):
        super().__init__(*args)
        self.forward_grads = {self: 1}

# Define our inputs as ForwardValue objects
a = ForwardValue(5)
b = ForwardValue(2)

# Perform operations
c = a * b
g = 3 * c + a

c.forward_grads = {a: 2, b: 5}  # dc/da and dc/db
g.forward_grads = {a: 3 * 2 + 1, b: 3 * 5} # dg/da = dg/dc dc/da + dg/da, dg/db = dg/dc dc/db

def forward_pass(self):
    # Clear forward_grads if it exists
    self.forward_grads = {} 
    
    grads = self.grads(*self.args)

    for i in range(len(self.parents)):
        if isinstance(self.parents[i], AutogradValue):
            for key, value in self.parents[i].forward_grads.items():
                update = value * grads[i]
                if key in self.forward_grads:
                    self.forward_grads[key] += update
                else:
                    self.forward_grads[key] = update

    # Make sure to still return the operation's value
    return self.func(*self.args)

# Overwrite the AutogradValue method so that operators still work
AutogradValue.forward_pass = forward_pass

In [ ]:
a = ForwardValue(5)
b = ForwardValue(2)
L = -log(5 *b + a)

dL_da = L.forward_grads[a]
dL_db = L.forward_grads[b]
print('dL/da = %.3f, dL/db = %.3f' % (dL_da, dL_db)) # dL/da = -0.067, dL/db = -0.333

In [ ]:
class ForwardModeNeuralNetwork(NeuralNetwork):

    def wrap_array_f(self, a):
        '''
        Wraps the elements of an array with AutogradValue

        Args:
            a (array of float): The array to wrap
        Returns:
            g (array of AutogradValue): An array g, such that g[i,j] = AutogradValue(a[i,j])
        '''
        g = []
        for i in a:
            g += [[ForwardValue(j) for j in i]]

        return np.array(g)

    def unwrap_gradient_f(self, L, a):
        '''
        Unwraps the gradient of an array with AutogradValues

        Args:
            a (array of AutogradValue): The array to unwrap
        Returns:
            g (array of float): An array g, such that g[i,j] = a[i,j].grad
        '''
        g = []
        for i in a:
            g += [[L.forward_grads[j] for j in i]]

        return np.array(g)
    

    def nll_and_grad(self, X, y):

        W = [self.wrap_array_f(w) for w in self.weights]

        L = self.nll(X, y, W)
        L.forward_pass()

        grad = [self.unwrap_gradient_f(L, w) for w in W]

        return L.value, grad  

In [ ]:
# Test forward

X, y = make_moons(100, noise=0.1)
model = ForwardModeNeuralNetwork(2, [5, 5])
gradient_descent(model, X, y, lr=3e-2, steps=250)

print('Model accuracy: %.3f' % model.accuracy(X, y))
plot_boundary(model, X, y)